## PySpark训练word2vec实现内容相似推荐

实现步骤：
1. 获取文章列表数据，包括ID、标题、内容
2. 使用jieba实现中文分词
3. 送入pyspark实现word2vec的训练，得到文章向量
4. 对于输入的ID，计算最相似的文章列表

### 1. 获取数据

In [1]:
import pandas as pd

In [2]:
csv_file = "./datas/ZhengFeixiang.csv"
csv_data = pd.read_csv(csv_file, low_memory = False)#防止弹出警告
df = pd.DataFrame(csv_data)

In [3]:
df.head(3)

,名字,账号,试卷,试题,对0错1,时间,详情,试卷名称,试题简介,章节
0,郑飞翔,202216012,5695,46499,0,20-09-07 10:27:23,e6d7733789671e1ea1ce5ff38cda6558,第1节 长度和时间的测量,(柳州中考)一个中学生的身高约为(),NaN
1,郑飞翔,202216012,5695,46500,0,20-09-07 10:27:24,be7b44771617bbdda1ce5ff38cda6558,第1节 长度和时间的测量,下列长度单位换算正确的是(),NaN
2,郑飞翔,202216012,5695,46502,1,20-09-07 10:27:25,f7b4315417d11ef3a1ce5ff38cda6558,第1节 长度和时间的测量,"初中生小红想对比测量一个月前后自己的身高变化,选用下列哪种尺最合适()",NaN


### 2. 使用jieba实现中文分词

pip install jieba

In [4]:
import jieba

In [5]:
def do_cut_words(param_df):
    # 标题加上关键词，是整个待分词的句子
    sentence = param_df["试卷名称"]+","+(param_df["试题简介"])
    # 调用分词
    words = list(jieba.cut(sentence))
    # 做过滤，变成小写
    result = []
    for word in words:
        if not word or len(word)==0 or len(word)==1: 
            continue
        word = word.lower()
        result.append(word)
    return " ".join(result)

df["words"] = df.apply(do_cut_words, axis=1)

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\ADMINI~1\AppData\Local\Temp\2\jieba.cache
Loading model cost 0.883 seconds.
Prefix dict has been built successfully.


In [6]:
#添加自增的序号列
df['id'] = range(len(df))

In [7]:
df.head(5)

,名字,账号,试卷,试题,对0错1,时间,详情,试卷名称,试题简介,章节,words,id
0,郑飞翔,202216012,5695,46499,0,20-09-07 10:27:23,e6d7733789671e1ea1ce5ff38cda6558,第1节 长度和时间的测量,(柳州中考)一个中学生的身高约为(),NaN,长度 时间 测量 柳州 中考 一个 中学生 身高,0
1,郑飞翔,202216012,5695,46500,0,20-09-07 10:27:24,be7b44771617bbdda1ce5ff38cda6558,第1节 长度和时间的测量,下列长度单位换算正确的是(),NaN,长度 时间 测量 下列 长度 单位 换算 正确,1
2,郑飞翔,202216012,5695,46502,1,20-09-07 10:27:25,f7b4315417d11ef3a1ce5ff38cda6558,第1节 长度和时间的测量,"初中生小红想对比测量一个月前后自己的身高变化,选用下列哪种尺最合适()",NaN,长度 时间 测量 初中生 小红想 对比 测量 一个月 前后 自己 身高 变化 选用 下列 哪...,2
3,郑飞翔,202216012,5695,46503,0,20-09-07 10:27:25,b9cb1cfbf6353064a1ce5ff38cda6558,第1节 长度和时间的测量,如图所示测量木块长度的方法中正确的是(),NaN,长度 时间 测量 如图所示 测量 木块 长度 方法 正确,3
4,郑飞翔,202216012,5695,46505,1,20-09-07 10:27:25,8e3e6b329f6b4ab3a1ce5ff38cda6558,第1节 长度和时间的测量,下列各个过程中经历的时间最接近1秒的是(),NaN,长度 时间 测量 下列 各个 过程 经历 时间 接近,4


In [11]:
# 保存成CSV
df[["id", "试卷名称","试题简介", "words"]].to_csv("./datas/questions_wordsegs.csv", index=False)

### 3. 使用pyspark训练word2vec

In [8]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("test pyspark") \
    .getOrCreate()

sc = spark.sparkContext

#### Pyspark读取CSV数据

In [9]:
df = spark.read.csv("./datas/questions_wordsegs.csv", header=True)
df.show(5)

+---+-----------------------+--------------------------------------+--------------------------------+
| id|               试卷名称|                              试题简介|                           words|
+---+-----------------------+--------------------------------------+--------------------------------+
|  0|第1节　长度和时间的测量|      (柳州中考)一个中学生的身高约为()|长度 时间 测量 柳州 中考 一个...|
|  1|第1节　长度和时间的测量|            下列长度单位换算正确的是()|长度 时间 测量 下列 长度 单位...|
|  2|第1节　长度和时间的测量| 初中生小红想对比测量一个月前后自己...|长度 时间 测量 初中生 小红想 ...|
|  3|第1节　长度和时间的测量|如图所示测量木块长度的方法中正确的是()|长度 时间 测量 如图所示 测量 ...|
|  4|第1节　长度和时间的测量|  下列各个过程中经历的时间最接近1秒...|长度 时间 测量 下列 各个 过程...|
+---+-----------------------+--------------------------------------+--------------------------------+
only showing top 5 rows



In [10]:
from pyspark.sql import functions as F
from pyspark.sql import types as T

In [11]:
# 把非常的字符串格式变成LIST形式
df = df.withColumn('words_split', F.split(df.words, " "))

#### 实现word2vec的训练与转换

In [12]:
# https://spark.apache.org/docs/2.4.6/ml-features.html#word2vec

from pyspark.ml.feature import Word2Vec

word2Vec = Word2Vec(
    vectorSize=5, 
    minCount=0, 
    inputCol="words_split", 
    outputCol="word2vec")

model = word2Vec.fit(df)

# 注意这一步，会得到整个doc的word embedding
df_word2vec = model.transform(df)

In [17]:
df_word2vec.printSchema()

root
 |-- id: string (nullable = true)
 |-- 试卷名称: string (nullable = true)
 |-- 试题简介: string (nullable = true)
 |-- words: string (nullable = true)
 |-- words_split: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- word2vec: vector (nullable = true)



In [13]:
df_word2vec.select("word2vec").show(3, truncate=False)

+----------------------------------------------------------------------------------------------------------+
|word2vec                                                                                                  |
+----------------------------------------------------------------------------------------------------------+
|[0.026515203338931315,-0.04343758086906746,0.11416542460210621,-0.25740674207918346,-0.014290090301074088]|
|[0.33545571885770187,0.05763895798008889,-0.03528693271800876,-0.22486617788672447,-0.08721717586740851]  |
|[0.11134966773533961,0.0045482304121833295,0.028219142404850572,-0.21513917623087764,0.028546714427648112]|
+----------------------------------------------------------------------------------------------------------+
only showing top 3 rows



In [14]:
df_word2vec.select("id", "试卷名称","试题简介", "word2vec") \
           .toPandas() \
           .to_csv('./datas/questions_words_word2vec.csv', index=False)

### 4. 对于给定文章算出最相似的10篇文章

In [15]:
df = pd.read_csv("./datas/questions_words_word2vec.csv")
df.head(3)

,id,试卷名称,试题简介,word2vec
0,0,第1节 长度和时间的测量,(柳州中考)一个中学生的身高约为(),"[0.026515203338931315,-0.04343758086906746,0.1..."
1,1,第1节 长度和时间的测量,下列长度单位换算正确的是(),"[0.33545571885770187,0.05763895798008889,-0.03..."
2,2,第1节 长度和时间的测量,"初中生小红想对比测量一个月前后自己的身高变化,选用下列哪种尺最合适()","[0.11134966773533961,0.0045482304121833295,0.0..."


In [16]:
import numpy as np
import json

In [17]:
df["word2vec"] = df["word2vec"].map(lambda x : np.array(json.loads(x)))

In [18]:
df.head(3)

,id,试卷名称,试题简介,word2vec
0,0,第1节 长度和时间的测量,(柳州中考)一个中学生的身高约为(),"[0.026515203338931315, -0.04343758086906746, 0..."
1,1,第1节 长度和时间的测量,下列长度单位换算正确的是(),"[0.33545571885770187, 0.05763895798008889, -0...."
2,2,第1节 长度和时间的测量,"初中生小红想对比测量一个月前后自己的身高变化,选用下列哪种尺最合适()","[0.11134966773533961, 0.0045482304121833295, 0..."


In [19]:
# 随便挑选一篇文章ID，2583：pandas，581：PHP
article_id = 258
df.loc[df["id"]==article_id]

,id,试卷名称,试题简介,word2vec
258,258,09月30日物理课堂练习201改练声音的产生,"(随州中考)如图所示,在探究“声音是由物体振动产生的”实验中,将正在发声的音叉紧靠悬线下的轻...","[-0.09066608338616788, -0.2947384475264698, 0...."


In [20]:
article_embedding = df.loc[df["id"]==article_id, "word2vec"].iloc[0]
article_embedding

array([-0.09066608, -0.29473845,  0.13494904, -0.10878635, -0.37069783])

In [21]:
# 余弦相似度
from scipy.spatial import distance
df["sim_value"] = df["word2vec"].map(lambda x : 1 - distance.cosine(article_embedding, x))

In [22]:
df[["id", "试题简介", "sim_value"]].head(10)

,id,试题简介,sim_value
0,0,(柳州中考)一个中学生的身高约为(),0.402605
1,1,下列长度单位换算正确的是(),0.021578
2,2,"初中生小红想对比测量一个月前后自己的身高变化,选用下列哪种尺最合适()",0.041274
3,3,如图所示测量木块长度的方法中正确的是(),0.218574
4,4,下列各个过程中经历的时间最接近1秒的是(),0.063433
5,5,"物理实验室中,常用的测量时间的工具是()",0.481784
6,6,"图为小组同学学习长度测量后讨论交流的情境,下列关于误差说法中正确的是()",-0.284397
7,7,"在测量物体长度时,由于下列原因造成测量结果有差异,其中属于误差的是()",0.300727
8,8,下列估计的数据与实际最接近的是(),-0.075197
9,9,"某同学测得物理课本长为25.91 cm,宽为18.35 cm,那么他所用刻度尺的分度值为()",0.174850


In [23]:
# 按相似度降序排列，查询前10条
df.sort_values(by="sim_value", ascending=False).head(20)

,id,试卷名称,试题简介,word2vec,sim_value
258,258,09月30日物理课堂练习201改练声音的产生,"(随州中考)如图所示,在探究“声音是由物体振动产生的”实验中,将正在发声的音叉紧靠悬线下的轻...","[-0.09066608338616788, -0.2947384475264698, 0....",1.000000
218,218,09月28日物理课堂练习59062音色专项,如图所示的三幅图分别是音叉、钢琴、长笛发出的声音在示波器上显示的波形.下面关于三者发出声音 ...,"[-0.01149562507867813, -0.2965747931599617, 0....",0.985087
202,202,09月28日物理课堂练习59062音色专项,如图所示的三幅图分别是音叉、钢琴、长笛发出的声音在示波器上显示的波形.下面关于三者发出声音 ...,"[-0.01149562507867813, -0.2965747931599617, 0....",0.985087
260,260,09月30日物理课堂练习201改练声音的产生,"(桂林中考)如图所示,把正在响铃的闹钟放在玻璃罩内,逐渐抽出罩内的空气,听到声音逐渐变小,再...","[-0.07913552398827892, -0.21575263894542498, 0...",0.982523
234,234,09月28日物理课堂练习203声音的利用,"(2019成都)如图所示为音叉共鸣实验:两个频率相同的音叉,用橡皮槌敲击其中一个音叉,另一个...","[-0.14672070218250155, -0.21696887597441675, 0...",0.982011
164,164,09月22日物理课堂练习声音的产生1,能说明“液体可以传播声音”的事例是( ),"[-0.04406843952495944, -0.45254255865107884, 0...",0.980962
210,210,09月28日物理课堂练习59070水瓶专项,"拿一张硬纸片让它快些从一木梳上划过,再让它从同一木梳上慢一点划过,则两次所发出的声音( )","[-0.023115277150645853, -0.3588521210709587, 0...",0.980106
188,188,09月28日物理课堂练习59070水瓶专项,"拿一张硬纸片让它快些从一木梳上划过,再让它从同一木梳上慢一点划过,则两次所发出的声音( )","[-0.023115277150645853, -0.3588521210709587, 0...",0.980106
223,223,09月28日物理课堂练习59062音色专项,"在二胡表演时,表演者在用弓拉动琴弦的同时,不断用手指去控制琴弦的长度,这样做是为了改变声音 ...","[-0.019112221494486388, -0.4788509077519517, 0...",0.977777
201,201,09月28日物理课堂练习59062音色专项,"在二胡表演时,表演者在用弓拉动琴弦的同时,不断用手指去控制琴弦的长度,这样做是为了改变声音 ...","[-0.019112221494486388, -0.4788509077519517, 0...",0.977777
